## Day 15: Beacon Exclusion Zone

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2022/tree/HEAD/notebooks/day15/puzzle.ipynb)

### Parsing

In [23]:

#load "../common/common.fsx"

type CoveredArea = { N : Point; E : Point; S : Point; W : Point }
type SensorData = { Beacon: Point; Sensor : Point; Covered : CoveredArea }

let read s = 
    let getCoveredArea (beacon : Point) (sensor : Point) = 
        let d = Point.mlen beacon sensor
        {N = sensor - Point(0, d); S = sensor + Point(0, d);
            E = sensor + Point(d, 0); W = sensor - Point(d, 0)}
    match s with 
    | Regex "Sensor at x=([-\d]+), y=([-\d]+): closest beacon is at x=([-\d]+), y=([-\d]+)" 
        [sensorx; sensory; beaconx; beacony] ->
            let beacon = Point(int beaconx, int beacony)
            let sensor = Point(int sensorx, int sensory)
            {Sensor = sensor; Beacon = beacon; Covered = getCoveredArea beacon sensor }
    | _ -> failwith ""
    


In [24]:
#!value --name sampleRaw
Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3

In [25]:
#!share sampleRaw --from value
let sampleData = Pattern1.read read sampleRaw
sampleData

index Beacon Sensor Covered 0 Item ( -2, 15 ) Item ( 2, 18 ) N E S W Point
 Item: ( 2, 11 ) Point
 Item: ( 9, 18 ) Point
 Item: ( 2, 25 ) Point
 Item: ( -5, 18 ) 1 Item ( 10, 16 ) Item ( 9, 16 ) N E S W Point
 Item: ( 9, 15 ) Point
 Item: ( 10, 16 ) Point
 Item: ( 9, 17 ) Point
 Item: ( 8, 16 ) 2 Item ( 15, 3 ) Item ( 13, 2 ) N E S W Point
 Item: ( 13, -1 ) Point
 Item: ( 16, 2 ) Point
 Item: ( 13, 5 ) Point
 Item: ( 10, 2 ) 3 Item ( 10, 16 ) Item ( 12, 14 ) N E S W Point
 Item: ( 12, 10 ) Point
 Item: ( 16, 14 ) Point
 Item: ( 12, 18 ) Point
 Item: ( 8, 14 ) 4 Item ( 10, 16 ) Item ( 10, 20 ) N E S W Point
 Item: ( 10, 16 ) Point
 Item: ( 14, 20 ) Point
 Item: ( 10, 24 ) Point
 Item: ( 6, 20 ) 5 Item ( 10, 16 ) Item ( 14, 17 ) N E S W Point
 Item: ( 14, 12 ) Point
 Item: ( 19, 17 ) Point
 Item: ( 14, 22 ) Point
 Item: ( 9, 17 ) 6 Item ( 2, 10 ) Item ( 8, 7 ) N E S W Point
 Item: ( 8, -2 ) Point
 Item: ( 17, 7 ) Point
 Item: ( 8, 16 ) Point
 Item: ( -1, 7 ) 7 Item ( 2, 10 ) Item ( 2, 0 ) N E S W Point
 Item: ( 2, -10 ) Point
 Item: ( 12, 0 ) Point
 Item: ( 2, 10 ) Point
 Item: ( -8, 0 ) 8 Item ( 2, 10 ) Item ( 0, 11 ) N E S W Point
 Item: ( 0, 8 ) Point
 Item: ( 3, 11 ) Point
 Item: ( 0, 14 ) Point
 Item: ( -3, 11 ) 9 Item ( 25, 17 ) Item ( 20, 14 ) N E S W Point
 Item: ( 20, 6 ) Point
 Item: ( 28, 14 ) Point
 Item: ( 20, 22 ) Point
 Item: ( 12, 14 ) 10 Item ( 21, 22 ) Item ( 17, 20 ) N E S W Point
 Item: ( 17, 14 ) Point
 Item: ( 23, 20 ) Point
 Item: ( 17, 26 ) Point
 Item: ( 11, 20 ) 11 Item ( 15, 3 ) Item ( 16, 7 ) N E S W Point
 Item: ( 16, 2 ) Point
 Item: ( 21, 7 ) Point
 Item: ( 16, 12 ) Point
 Item: ( 11, 7 ) 12 Item ( 15, 3 ) Item ( 14, 3 ) N E S W Point
 Item: ( 14, 2 ) Point
 Item: ( 15, 3 ) Point
 Item: ( 14, 4 ) Point
 Item: ( 13, 3 ) 13 Item ( 15, 3 ) Item ( 20, 1 ) N E S W Point
 Item: ( 20, -6 ) Point
 Item: ( 27, 1 ) Point
 Item: ( 20, 8 ) Point
 Item: ( 13, 1 )

In [26]:
#!value --name actualRaw --from-file ./data_actual.txt

In [27]:
#!share actualRaw --from value
let actualData = Pattern1.read read actualRaw

### Part 1

In [28]:
let segment atY area = 
    if (atY <= Point.y area.N || atY >= Point.y area.S)
    then None
    else 
        let d = atY - Point.y area.W
        Range.T (Point.x area.W + abs d, Point.x area.E - abs d)
        |> Some

In [41]:
let countCovered atY (data : SensorData[]) = 
    let allBeaconsAtY = 
        data 
        |> Seq.map (fun d -> d.Beacon)
        |> Seq.distinct
        |> Seq.filter (fun b -> atY = Point.y b)
        |> Array.ofSeq

    let atYsegments = 
        data
        |> Array.choose (fun sd -> segment atY sd.Covered (*|> Option.map (fun x -> sd,x)*))
    
    let segmentsNormalized = 
        atYsegments |> Array.toList |> Range.unionAll

    let beaconsAtSegments = 
        allBeaconsAtY
        |> Array.filter (fun b -> segmentsNormalized |> List.exists (Range.contains (Point.x b)))

    (segmentsNormalized
    |> Seq.map (Range.len)
    |> Seq.sum)
    - 
    Array.length beaconsAtSegments


    

In [42]:
countCovered 10 sampleData

26

In [43]:
countCovered 2_000_000 actualData

5525990

### Part 2

In [57]:
let onlyUncoveredBeacon (xRange : Range.T) (yRange : Range.T) (data : SensorData[]) = 
    let (yFound, [segment2; segment1]) = 
        seq {
            for y = Range.start yRange to Range.finish yRange do
                let segments = 
                    data
                    |> Seq.choose (fun data -> segment y data.Covered)
                    |> Seq.filter (fun r -> Range.overlap r xRange)
                    |> Seq.toList
                    |> Range.unionAll
                yield y,segments

        } |> Seq.filter (snd >> List.length >> ((=)2))
        |> Seq.exactlyOne
    Point (Range.finish segment1 + 1, yFound)

let tuningFrequency beacon = 
    int64 (Point.x beacon) * 4_000_000L + int64 (Point.y beacon)

In [58]:
sampleData |> onlyUncoveredBeacon (0, 20) (0, 20)
|> displayPipe |> tuningFrequency

Item
"( 14, 11 )"


56000011

In [61]:
actualData |> onlyUncoveredBeacon (0, 4_000_000) (0, 4_000_000)
|> displayPipe |> tuningFrequency

Item
"( 2939043, 2628223 )"


11756174628223